## Some common gotchas

### Equations with constant terms (`BoundsError`)

Equations that involve constant terms may give a `BoundsError`. This is a known bug (cf. issue [#179](https://github.com/JuliaReach/ReachabilityAnalysis.jl/issues/179)]) and it related to cases in which the update rule for the right-hand side does not have the same coefficient types. Suppose that we have a varable `u[1]` whose dynamics is `du[1] = 1.0`; this likely fails in the integration (see the linked issue). However, we can write `du[1] = 1.0 + one(u[1])`, or store the constant term in an auxiliary variable as shown in the next example.

**Example.** Consider the function `f!`: 

```julia
@taylorize function f!(du, u, p, t)
    du[1] = u[3]^3 - u[2] + u[4]
    du[2] = u[3]
    du[3] = 2.0
    du[4] = u[4]
    return du
end
```

This function can be re-written like so:

```julia
@taylorize function f!(du, u, p, t)
    local two = 2.0 + zero(u[1])
    du[1] = u[3]^3 - u[2] + u[4]
    du[2] = u[3]
    du[3] = two
    du[4] = u[4]
    return du
end
```

### What is `@taylorize`? Do I need it?

`@taylorize` is a macro which parses the functions containing the ODEs to be integrated, allowing to speed up repeated evaluations. The macro is defined in [TaylorIntegration.jl](https://perezhz.github.io/TaylorIntegration.jl), see [`@taylorize`'s documentation in TaylorIntegration.jl](https://perezhz.github.io/TaylorIntegration.jl/stable/taylorize/) for further details. Since it is an optimization, it is *not* mandatory, though it is recommended as it helps to reduce the number of allocations and as a consequence it usually gives a performance boost.

### How can I get the most of out `@taylorize`?

The main advice is to write expressions that invovle two arguments at most, making use of parentheses to write expressions which involve more than two terms. For further limitatins and comments see [this section of TaylorInegrations.jl's documentation](https://perezhz.github.io/TaylorIntegration.jl/stable/taylorize/#Limitations-and-some-advices-1).

**Example.** Here is an example that uses some of the above recommendations. Start with `f!` defined below:

```julia
@taylorize function f!(du, u, params, t)
    local a = 0.3
    x, y, z = u[1], u[2], u[3]
    
    du[1] = -x * y/(1 + x)
    du[2] = x * y/(1 + x) - a * y
    du[3] = a * y
    return du
end
```

Observe that the terms `x * y` can be factored out, and we can use an auxiliary variable `aux` such that all intermediate operations involve only two terms like so:

```julia
@taylorize function g!(du, u, params, t)
    local a = 0.3
    x, y, z = u[1], u[2], u[3]
    
    num = x * y
    den = 1 + x
    aux = num/den
    du[1] = -aux
    du[2] = aux - a * y
    du[3] = a * y
    return du
end
```

### Why I can't directly plot the solution?

The flowpipe computed with algorithm `TMJets` is a *Taylor model flowpipe*, and to reason about such flowpipes, either for plotting or performing set-based operations, we *overapproximate* them with other set representations (usually, convex sets such as boxes or polytopes). The command `overapproximate(sol, Zonotope)` applies `overapproximate(Ri, Zonotope)` for each reach-set `Ri` in the solution.

-----

**Running example.** Our running example is a [damped harmonic oscillator](https://en.wikipedia.org/wiki/Harmonic_oscillator), whose dynamics is 

$x''(t) = -\omega^2 x(t)$

In [ ]:
https://en.wikipedia.org/wiki/Harmonic_oscillator

We can turn this second order system into a first order one introducing the state variables $(x, v)$; the equations are



In [41]:
ω₀ = 1
ζ = 1/2

0.5

In [42]:
A = [0          1;
    -ω₀^2   -2*ζ*ω₀]

2×2 Array{Float64,2}:
  0.0   1.0
 -1.0  -1.0

In [45]:
S = @system(x' = Ax)

LinearContinuousSystem{Float64,Array{Float64,2}}([0.0 1.0; -1.0 -1.0])

## Set representations

A *half-space* $H \subseteq \mathbb{R}^n$ is the set of points satisfying the linear constraints

$$
H = \{ x : a^T x \leq b\}
$$
For example, the half-space $x + y \leq 2$ we defined with

```julia
julia> H = HalfSpace([1.0, 1.0], 2.0)
```

A *polyhedron* $P$ is the intersection of a finite number of half-spaces,

$$
P = \left\{ x \in \mathbb{R}^n: \bigcap_{i=1}^m a_i^T x \leq b_i\right\}
$$

where $a_i \in \mathbb{R}^n$ and $b_i \in \mathbb{R}$.

A *polytope* is a bounded polyedron. The convex hull of a set $X \subseteq \mathbb{R}^n$ is

$$
CH(X)= \left\{\sum_{i=1}^m \lambda_i v_i : v_i \in X, \lambda_i \geq 0, \sum_{i=1}^m \lambda_i = 1\right\}.
$$

Given a matrix $M \in \mathbb{R}^{n\times n}$, $MX = \{Mx : x \in X\}$. The *Minkowski sum* of sets $X$ and $Y$ is 

## Reach-sets

## Flowpipes

A *flowpipe* is the union of reach-sets.